<a href="https://colab.research.google.com/github/sudhir2016/India_climate_map/blob/master/India_climate_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Geopandas,Rasterio and rioxarray.

In [ ]:
!pip install geopandas

In [ ]:
!pip install rasterio

In [ ]:
!pip install rioxarray

Import various libraries.

In [80]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona as fn
import rasterio as rio
import xarray as xa
import rioxarray as rioxa
from rasterio import mask
from sklearn.cluster import KMeans

Mount Google Drive as the climate data is saved there.

In [81]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Load climate data.

In [83]:
data='/content/drive/My Drive/adaptor.mars.internal-1599570962.375764-24582-1-fe6db266-a043-47ab-b3c2-5f6f4072408a.nc'

Open dataset using Xarray.

In [84]:
x=xa.open_dataset(data)

Set crs as epsg 4326.

In [ ]:
x.rio.set_crs('epsg:4326')

Create and open tif files for precipitation and temp.

In [86]:
pays='india'

In [87]:
x['tp'].rio.to_raster('{}_tp.tif'.format(pays))

In [88]:
x['t2m'].rio.to_raster('{}_t2m.tif'.format(pays))

In [89]:
tp=rio.open('{}_tp.tif'.format(pays))

In [90]:
t2m=rio.open('{}_t2m.tif'.format(pays))

Read India Geojason file and use it to mask the precipitation and temp data.

In [91]:
india='/content/India.json'

In [92]:
map=gpd.read_file(india)

In [ ]:
map.plot()

In [94]:
with fn.open(india,'r') as shapefile:
  shapes=[feature['geometry'] for feature in shapefile]

In [95]:
tp,tp_transform=rio.mask.mask(tp,shapes,crop=True)

In [96]:
t2m,t2m_transform=rio.mask.mask(t2m,shapes,crop=True)

Extract precipitation and temp data as array and reshape it to suit KMeans.

In [97]:
l=tp.shape[0]
h=tp.shape[1]
w=tp.shape[2]
print(l,h,w)

474 276 259


In [98]:
c=np.zeros((h,w,2*l))

In [99]:
for j in range (h):
  for k in range (w):
    c[j,k,:l]=tp[:,j,k]
    c[j,k,l:]=t2m[:,j,k]

In [100]:
img=c[:,:,:].reshape((c.shape[0]*c.shape[1],c.shape[2]))

In [ ]:
img.shape

Run KMeans with number of clusters as 6.

In [102]:
k=KMeans(n_clusters=5)

In [103]:
y=k.fit_predict(img)

In [ ]:
k.cluster_centers_

Extract labels, reshape and visualize.

In [ ]:
lab=k.labels_
print(lab)

In [107]:
lab=lab.reshape(( c[:,:,:].shape[0],c[:,:,:].shape[1]))

In [120]:
cmap = mpl.colors.ListedColormap(['w', 'g','#D3D3D3','b','c'])

In [ ]:
plt.imshow(lab,cmap=cmap)